<H1> The punch you don’t see is the punch that hurts</H1>
 > heavyweight boxer Willie Pastrano (1935-1997)

# Overview

   Known potentially hazardous asteroids (PHAs) are tracked by a world-wide volunteer network of optical telescopes organized by ESA and JPL.  This PHAs observing service works for objects that are known and that due to combination of size, proximity and surface characteristics have a sufficiently high optical albedo to be optically detectable.  The analysis here presents an initial feasibility assessment of a radio frequency (RF) sensor network to detect unknown NEOs and PHAs as well as track known PHAs from the night side of earth (NSoE) using a reference signal from the day side of earth(DSoE).  One of the LibreSpace projects, SATNOGS, interconnects a world-wide network of satellite signal tracking enthusiasts with antennas of modest capabilities.  The analysis will pose and answer the question of what is the sized of a network of "enthusiast-grade" [1](#Footnotes) networked radio telescopes that could detect PHAs in the range not visible by ESA and JPL observing volunteers.  
   

# Background 

   The network would use DSoE and NSoE signals to develop passive radar track on NSoE near-earth objects (NEOs) that cross or reside outside of Earth Orbit. NEOs are classified in one of four classes according to their perihelion distance (q), aphelion distance (Q) and their semi-major axes (a): Atens, Apollos, Amors and Atiras[4]. The analysis here examines addresses the former three classes(note 1) because of their orbits location with respect to the earth.  The reference signal would be developed based on specific discrete solar RF radiation sources as isotropic radiators.

   notes:
   1. Atiras (a < 1 AU, Q < 0.983 AU) don't ever cross outside earth orbit and wouldn't be detectable through backscatter. Forward scatter investigation could complement this work at a later time.

# Geometry
   * <img src='astroid_detection_geometry.svg' width="600" height="400">

# Footnotes
A. <$1000 total cost, < 2Ghz Center Frequency, < 2 MHz BW, Az/El Rotator, < 2 Mbps Internet uplink, GPS Disciplined Oscillator 


# References
1. [H. D. Griffiths, Bistatic and Multistatic Radar](http://www.ittc.ku.edu/~callen/energy_harvesting/Griffiths2003pp9.pdf)
2. [M.K. Shepard et al., Radar observations and shape model of asteroid 16 Psyche, Icarus (2016)](http://dx.doi.org/10.1016/j.icarus.2016.08.011)
3. [Near Earth Objects - Dynamic Site](https://newton.spacedys.com/neodys/)
4. [Center for Near Earth Objects Website](https://cneos.jpl.nasa.gov/about/neo_groups.html)
5. [W. N. CHRISTIANSEN et al., The Distribution of Radio Brightness over the Solar Disat at a Wavelength of 21 Centimeters]()

In [2]:
import numpy as np

#System-wide Description
Terr = 50e-6           # seconds    
print(f"Multi-Antenna System Parameters: \n\tSite to Site Syncronization:{Terr * 1e6:.1f} microseconds")
print("\t\tstate of the art for 2022 using prosumer-grade \n\t\tGNSS/GPS TXCO master oscillator")

#Receiver Params
#Receiver System Calculations	Variable	Units
#RX antenna gain	GR	dBi
GR = 30                #dBi

#RX connector loss	LConR1	dB
LConR1 = 2             #dB

#RX cable loss	LCabR	dB
LCabR = 2             #dB

#RX connector loss (remote antenna)	LConR2	dB
LConR2 = 2             #dB

#RX Sensitivity  Prxs  dBm
Prxs = -115            #dBm

print(f"Antenna System Calcs: \n\tAntenna Gain: {GR} dBi")
print(f"\tCable & Connector Loss: {LConR1+LCabR+LConR2} dB")
print(f"\tRX Sensitivity  Prxs  {Prxs} dBm")

## SENSOR PARARMS
#Receiver Sensitivity Calculations	
#RX Noise Figure	NF
NF=5                    #dB

#Operating Temperature	T0
T0 = 290                #°K

#Effective Noise Temperature	Te
#  Te = T0(NF - 1)     
Te =T0*(np.power(10,(NF/10))-1)

#Boltzmann's constant	k
k=1.38E-23             #J/K

#Receive Bandwidth	BWRX
BWRX = 10              #MHz

#Antenna Temperature	TAnt
Tant = 300             #°k

#Noise Power (at RX)	Pn
# Pn = k(TAnt + Te)BWRX
Pn = k * (Tant +Te) * BWRX
Pn =10*np.log10(k*(Tant+Te)*(BWRX*1000000))+30

#Required Signal to Noise Ratio	SNRRX
SNRRX = Prxs-Pn

print(f"Rx Sensitivt Calcs:\n\tNoise Figure: {NF} dB")
print(f"\tTemperatures  Operating(To): {T0} °K  \tEffective(Te): {Te:.0f} °K  \tAntenna(Tant): {Tant} °K")
print(f"\tReceive Bandwidth	(BWRX): {BWRX} MHz \tNoise Power (at RX) : {Pn:.1f} dB")
print(f"\tRequired SNR: {SNRRX:.1f} dB")

#Passive Illumination Source

Multi-Antenna System Parameters: 
	Site to Site Syncronization:50.0 microseconds
		state of the art for 2022 using prosumer-grade 
		GNSS/GPS TXCO master oscillator
Antenna System Calcs: 
	Antenna Gain: 30 dBi
	Cable & Connector Loss: 6 dB
	RX Sensitivity  Prxs  -115 dBm
Rx Sensitivt Calcs:
	Noise Figure: 5 dB
	Temperatures  Operating(To): 290 °K  	Effective(Te): 627 °K  	Antenna(Tant): 300 °K
	Receive Bandwidth	(BWRX): 10 MHz 	Noise Power (at RX) : -98.9 dB
	Required SNR: -16.1 dB


In [4]:
# OC radar albedo  (sigmaHatOC)
# ratio of its OC radar cross section (σOC) to its cross-sectional area - See Ref 2
#                4σOC 
# sigmaHatOC = -----------
#               π Deff^2
# assume a conservative value of 0.1
sigmaHatOC=0.1


In [6]:
#TargetSize
SS=10                 #sqm
dSS=10*np.log10(SS)
#Center Freq of Sample Signal
Fc=1.7               #GHz
#Speed of Light
c=299792458           #m/s
#Wavelength
Ld=c/(Fc*1e9)
#Incident Quiet Sun Flux on Target
# Fcenter - 
FLUXsunq=(Fc-BWRX/2/1000)*307555+16773
#Effective Radiated Power Spectrum

#Effective Radiated Power over Rx BW

print(f"dSS: {dSS} dBsm")
print(f"Wavelength (Lambda - Ld): {Ld} m")
print(f"Quiet Sun Flex Incident power {FLUXsunq}")


dSS: 10.0 dBsm
Wavelength (Lambda - Ld): 0.17634850470588234 m


In [130]:
180/np.pi

57.29577951308232

# Useful Quantities

 1. Irradiance Units [wikiPedia Ref Link](https://en.wikipedia.org/wiki/Irradiance)
    1. Spectral Radiance: watt per steradian per square metre per hertz 
        Radiance of a surface per unit frequency or wavelength. The latter is commonly measured in W⋅sr−1⋅m−2⋅nm−1. This is a directional quantity.
    2. Spectral intensity: watt per steradian per hertz 
        Radiant intensity per unit frequency or wavelength. The latter is commonly measured in W⋅sr−1⋅Hz-1. This is a directional quantity.  
2. Energy Units
   1. Jansky: unit of spectral flux density
        Pdbm*m^-2*Hz^-1 = 10*log10(Pjy)-230
        PdbW*m^-2*Hz^-1 = 10*log10(Pjy)-260
   2. erg/s to Watts: 1 W = 10000000 erg/s